In [1]:
import sys; sys.path.append('..'); sys.path.append('../..')
from common.postgresql import PostgresConnector
import pandas as pd
import re

db = PostgresConnector()
read_sql = db.read_sql
sql = db.execute_sql

# Leitura de Dados
Lendo gestores da CVM. A query foi construída para facilitar a inserção de novas tabelas via UNION.

In [2]:
query = """
SELECT DISTINCT gestor, 'cvm.cadastro' as tabela 
FROM cvm.cadastro 
WHERE dt_fim IS NULL AND pf_pj_gestor = 'PJ'
"""
df_gestores = read_sql(query)

# Definição de Padrões (De-Para)
Lista de tuplas `(Grupo, Regex Pattern)`. 
**Regra para nomes curtos (< 4 chars):** São tratados como palavras inteiras (match exato ou cercado por espaços) para evitar falsos positivos (ex: 'BB' não deve pegar 'BBA').

In [3]:
import re

patterns = []

with open('gestores.md', 'r', encoding='utf-8') as f:
    for line in f:
        if not line.strip() or line.startswith('#'):
            continue

        parts = line.split('\t')
        if len(parts) >= 2:
            # Captura exatamente a estrutura ('Grupo', r'Regex')
            match = re.search(r"\('([^']*)',\s*r'([^']*)'\)", parts[1])
            if match:
                patterns.append((match.group(1), match.group(2)))

In [4]:
def aplicar_depara(nome, patterns_list):
    if not isinstance(nome, str):
        return None
        
    # Normalização básica para ajudar no match (opcional, dependendo do regex)
    # nome_upper = nome.upper()
    
    for grupo, pattern in patterns_list:
        # Tratamento para nomes curtos (< 4 chars)
        regex_pattern = pattern
        if len(pattern) < 4:
            # Garante que é uma palavra sozinha: Começo de string ou espaço + pattern + Final de string ou fim de palavra/espaço
            # Usando word boundaries \b é bom, mas para caracteres acentuados ou específicos, o controle manual de espaço é mais seguro conforme pedido.
            # Pedido: "ou sozinho ou com um espaço"
            regex_pattern = r'(?:^|\s)' + pattern + r'(?:$|\s)'
        
        # Busca case insensitive
        if re.search(regex_pattern, nome, re.IGNORECASE):
            return grupo
            
    return None # Ou retornar 'Outros' / o próprio nome se preferir

# Testes rápidos de sanidade
testes = [
    ("Itaú Unibanco", "Itaú"),
    ("Itaú BBA", "Itaú"),
    ("BB Gestão", "BB"),
    ("Prev BB", "BB"),
    ("UBBA", None), # Não deve pegar BB
    ("BBA", None), # Não deve pegar BB (patterns são cases sensiveis? regex no codigo acima é IGNORECASE)
]

print("Validando lógica em casos de teste:")
for entrada, esperado in testes:
    resultado = aplicar_depara(entrada, patterns)
    match_ok = (resultado == esperado) or (esperado is None and resultado is None)
    print(f"'{entrada}' -> {resultado} (Esperado: {esperado}) [{'OK' if match_ok else 'FALHA'}]")

Validando lógica em casos de teste:
'Itaú Unibanco' -> Itaú (Esperado: Itaú) [OK]
'Itaú BBA' -> Itaú (Esperado: Itaú) [OK]
'BB Gestão' -> BB (Esperado: BB) [OK]
'Prev BB' -> BB (Esperado: BB) [OK]
'UBBA' -> None (Esperado: None) [OK]
'BBA' -> None (Esperado: None) [OK]


# Aplicação no Dataset

In [5]:
import numpy as np
df_gestores['grupo'] = df_gestores['gestor'].apply(lambda x: aplicar_depara(x, patterns))

# Filtrar ou organizar resultado final
# O usuário pediu colunas: "grupo", "gestor", "tabela"
df_final = df_gestores[['grupo', 'gestor', 'tabela']].copy()
df_final['grupo'] = np.where(df_final['grupo'].notna(), df_final['grupo'], df_final['gestor'])

# Mostrar exemplos classificados
print("\nExemplos classificados:")
display(df_final[df_final['grupo'].notnull()].head(10))

# Mostrar exemplos NÃO classificados (para aprimorar os patterns)
print("\nExemplos NÃO classificados:")
display(df_final[df_final['grupo'].isnull()].head(10))


Exemplos classificados:


,grupo,gestor,tabela
0,10B,10B GESTORA DE RECURSOS LTDA.,cvm.cadastro
1,23S,23S CAPITAL LTDA.,cvm.cadastro
2,2B,2B CAPITAL S.A.,cvm.cadastro
3,3 Ilhas,3 ILHAS INVESTIMENTOS - GESTÃO DE RECURSOS LTDA.,cvm.cadastro
4,3G,3G CAPITAL GESTORA DE RECURSOS LTDA,cvm.cadastro
5,3J,3J GESTORA DE RECURSOS LTDA. - ME,cvm.cadastro
6,3Q,3Q ASSET LTDA,cvm.cadastro
7,3R,3R GESTORA DE RECURSOS LTDA,cvm.cadastro
8,3V,3V CAPITAL GESTÃO DE RECURSOS LTDA.,cvm.cadastro
9,4I,4I CAPITAL LTDA,cvm.cadastro



Exemplos NÃO classificados:


,grupo,gestor,tabela


In [6]:
# df_final.to_sql('cvm_gestores', con=engine, if_exists='replace', index=False)
db.overwrite_table(df_final, 'cvm.depara_gestores')

Tabela cvm.depara_gestores sobrescrita com sucesso (Backup destruído).


In [7]:
import re
re.findall(r'B.{0,3}SIDE', 'B.SIDE WEALTH MANAGEMENT GESTÃO DE RECURSOS LTDA')

['B.SIDE']